In [4]:
import numpy as np
import os
from PIL import Image

from sklearn.metrics import pairwise_distances
homePath = r'E:\pycharm files\基于PCA的人脸识别\\'

In [5]:
def CreateDatabase(TrainDatabase):
    ############# File management
    TrainFiles = os.listdir(TrainDatabase)
    Train_Number = 0;
    # 统计训练图片数量
    for i in range(len(TrainFiles)):
        if TrainFiles[i]!='Thumbs.db':
            Train_Number = Train_Number + 1
    T = []
    # 按顺序读取文件
    for i in range(1,Train_Number+1):
        str_ = TrainDatabase + str(i) + ".jpg"
        image = Image.open(str_)
        img = list(image.getdata())
        T.append(img)
    T = np.array(T).T
    return T

def EigenfaceCore(T):
    m = T.mean(axis=1).reshape(-1, 1)  # Computing the average face image m = (1/P)*sum(Tj's)    (j = 1 : P)
    # Train_Number = T.shape(1)

    A = T - m  # all centered images

    L = np.matmul(A.T, A)  # L is the surrogate of covariance matrix C=A*A'.
    values, vectors = np.linalg.eig(L)  # Diagonal elements of D are the eigenvalues for both L=A'*A and C=A*A'.

    L_eig_vec = []
    for i in range(len(values)):
        if (values[i] > 1):
            L_eig_vec.append(vectors[:, i])

    L_eig_vec = np.array(L_eig_vec).T
    Eigenfaces = np.matmul(A, L_eig_vec)

    return m, A, Eigenfaces
def Recognition(TestImage, m, A, eigenfaces):
    Train_Number = eigenfaces.shape[1]
    # 将二十张训练图（centered image）都映射到19维的特征空间中
    ProjectedImages = np.matmul(eigenfaces.T, A)
    # 读取测试图片
    image = Image.open(TestImage)
    InImage = np.array(image.getdata()).reshape(-1, 1)
    # 归一化/中心化
    Difference = InImage - m

    # 将测试图片（centered image）映射到19维的特征空间中
    ProjectedTestImage = np.matmul(eigenfaces.T, Difference)

    # 计算测试特征图到每一张训练特征图欧氏距离
    Euc_dist = []
    for i in range(Train_Number):
        q = ProjectedImages[:, i].reshape(1, -1)
        twoPoints = np.vstack((ProjectedTestImage.T, q))
        temp = pairwise_distances(twoPoints, metric='euclidean')[1, 0]
        Euc_dist.append(temp)
    out_idx = np.argmin(np.array(Euc_dist)) + 1
    return out_idx

In [ ]:


from CreateDatabase import CreateDatabase
from EigenfaceCore import EigenfaceCore
from Recognition import Recognition

homePath = r'E:\pycharm files\基于PCA的人脸识别\\'
TrainDatabase = r"TrainDatabase\\"
T = CreateDatabase(homePath+TrainDatabase)
m,A,Eigenfaces = EigenfaceCore(T)

# 输入测试图片
TestImage = input('Enter test image name (a number between 1 to 10): ')
TestImage = 'TestDatabase\\' + TestImage + '.jpg'
OutputName=Recognition(homePath+TestImage, m, A, Eigenfaces)


# 绘制测试图片
image_test = Image.open(homePath+TestImage)
image_test.show()

# 绘制匹配到的图片
SelectedImage = homePath + r"TrainDatabase\\" +str(OutputName) + ".jpg"
print("Matched image is:",str(OutputName)+".jpg")
image_select = Image.open(SelectedImage)
image_select.show()